In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from scipy.cluster.hierarchy import dendrogram, linkage
from scipy.cluster import hierarchy
import gensim
from sklearn.cluster import MiniBatchKMeans
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score
from nltk.corpus import stopwords
from nltk import word_tokenize
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from sklearn.cluster import AgglomerativeClustering
#import scipy.cluster.hierarchy as hierarchy

In [ ]:
#import data

#posts = pd.read_csv(r"C:\...\2022-07-28-00-31-22-SGT-search-csv-export_lonely_loneliness_2020.csv")
posts = pd.read_csv(r"C:\...\2022-07-28-00-32-18-SGT-search-csv-export_lonely_loneliness_2021.csv")
#posts = pd.read_csv(r"C:\...\2022-07-28-00-34-46-SGT-search-csv-export_lonely_loneliness_2022.csv")

In [ ]:
posts['Page Description']

In [ ]:
# Data Preparation

def get_tokens(x):
    tokens = word_tokenize(str(x))
    return tokens

def no_stops(x):
    #en_stops = set(stopwords.words('taglish'))
    en_stops = set(stopwords.words('english'))
    no_stops = ''
    for word in get_tokens(x):
        if word not in en_stops:
            if len(word) > 3:
                no_stops += word + ' '
    return no_stops

In [ ]:
# Data Cleansing

clean_posts = pd.DataFrame(posts['Page Description'].apply(no_stops))
clean_posts['Page Description'] = clean_posts['Page Description'].str.replace('[^\w\s]',' ')
clean_posts['Page Description'] = clean_posts['Page Description'].str.replace('[0-9]',' ')
clean_posts['Page Description'] = clean_posts['Page Description'].str.lower()
clean_posts['Page Description'] = clean_posts['Page Description'].str.replace('facebook',' ')
clean_posts['Page Description'] = clean_posts['Page Description'].str.replace('philippine',' ')
clean_posts['Page Description'] = clean_posts['Page Description'].str.replace('philippines',' ')
clean_posts['Page Description'] = clean_posts['Page Description'].str.replace('www',' ')
clean_posts['Page Description'] = clean_posts['Page Description'].str.replace('post',' ')
clean_posts['Page Description'] = clean_posts['Page Description'].str.replace('photo',' ')
clean_posts['Page Description'] = clean_posts['Page Description'].str.replace('https',' ')
clean_posts['Page Description'] = clean_posts['Page Description'].str.replace('next',' ')
clean_posts['Page Description'] = clean_posts['Page Description'].str.replace('login',' ')
clean_posts['Page Description'] = clean_posts['Page Description'].str.replace('tinyurl',' ')
clean_posts['Page Description'] = clean_posts['Page Description'].str.replace('Singapore',' ')
clean_posts['Page Description'] = clean_posts['Page Description'].str.replace('singapore',' ')
clean_posts['Page Description'] = clean_posts['Page Description'].str.replace('official',' ')
clean_posts['Page Description'] = clean_posts['Page Description'].str.replace('page',' ')
clean_posts = pd.DataFrame(clean_posts['Page Description'].apply(no_stops))

In [ ]:
clean_posts

In [ ]:
vec1 = TfidfVectorizer(stop_words="english")
vec1.fit(clean_posts['Page Description'].values)
features1 = vec1.transform(clean_posts['Page Description'].values)

In [ ]:
# KMeans Clustering

def getscore(x, y):
    random_state = 2002
    km = MiniBatchKMeans(n_clusters=x, random_state = random_state)
    km.fit(y)
    return 'Score for ' + str(x) + ' clusters: ' + str(silhouette_score(km.fit_transform(y), km.labels_))

def getscores(x):
    response = []
    for i in range(3,30):
        response.append(getscore(i, x))
    return pd.DataFrame(response, columns=['outcomes'])

In [ ]:
getscores(features1)

In [ ]:
random_state = 2002
cls1 = MiniBatchKMeans(n_clusters=9, random_state=random_state)
cls1.fit(features1)

In [ ]:
# reduce the features to 2D
pca1 = PCA(n_components=2, random_state=random_state)
reduced_features1 = pca1.fit_transform(features1.toarray())

# reduce the cluster centers to 2D
reduced_cluster_centers1 = pca1.transform(cls1.cluster_centers_)

In [ ]:
plt.scatter(reduced_features1[:,0], reduced_features1[:,1], c=cls1.predict(features1))
plt.scatter(reduced_cluster_centers1[:, 0], reduced_cluster_centers1[:,1], marker='x', s=150, c='b')

In [ ]:
position1 = pd.DataFrame(reduced_features1, index=clean_posts.index)
position1.columns = ['x_position', 'y_position']

In [ ]:
labels1 = pd.DataFrame(cls1.labels_, index=clean_posts.index)
labels1.columns = ['label']

In [ ]:
clean_posts_labeled = clean_posts.join(labels1).join(position1)

In [ ]:
clean_posts_labeled

In [ ]:
def visualize_clouds(clusters, corpus):
    for x in range(clusters):
        topic = corpus['Page Description'][corpus.label == x]
        topics = ''
        for i in topic.values:
            topics += i
        #topics
        cloud = WordCloud(background_color='white', collocations=False).generate(topics)
        plt.figure(figsize=(10,6))
        plt.imshow(cloud, interpolation='bilinear')
        plt.axis("off")
        print("Topic #" + str(x))
        plt.show()
        #plt.savefig(f'wordcloud.png',
            #dpi = 300)

In [ ]:
#change based on number of clusters

visualize_clouds(9, clean_posts_labeled)

In [ ]:
def count_words(clusters, corpus):
    for x in range(clusters):
        topic = corpus['Page Description'][corpus.label == x]
        topics = ''
        for i in topic.values:
            topics += i

        #df=pd.DataFrame(r1,columns=['text'])
        #df.text.apply(lambda x: pd.value_counts(x.split(" "))).sum(axis = 0)
        print("Topic #" + str(x))
        df = pd.DataFrame([topics], columns=['text'])
        counts = df.text.apply(lambda x: pd.value_counts(x.split(" "))).sum(axis = 0)
        print(counts)
        pd.DataFrame(counts).to_csv('counts' + str(x) + '.csv')

In [ ]:
#change based on number of clusters

count_words(9, clean_posts_labeled)

In [ ]:
clean_posts_labeled[clean_posts_labeled['label'] == 1]